In [2]:
import numpy as np
import pandas as pd
import sagemaker
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

In [32]:
S3_BUCKET = "inflation-interest-unemployment"
FILENAME = "inflation interest unemployment.csv"
ROLE = sagemaker.get_execution_role()

In [4]:
!aws s3 cp "s3://$S3_BUCKET/$FILENAME" .

download: s3://inflation-interest-unemployment/inflation interest unemployment.csv to ./inflation interest unemployment.csv


In [5]:
df = pd.read_csv(FILENAME)

In [16]:
df.sample(10)

,country,year,"Inflation, consumer prices (annual %)","Inflation, GDP deflator (annual %)",Real interest rate (%),Deposit interest rate (%),Lending interest rate (%),"Unemployment, total (% of total labor force) (national estimate)","Unemployment, total (% of total labor force) (modeled ILO estimate)",iso3c,iso2c,adminregion,incomeLevel
8802,Myanmar,1984,4.846312,2.249637,NaN,1.500000,NaN,NaN,NaN,MMR,MM,East Asia & Pacific (excluding high income),Lower middle income
1295,Bermuda,2017,NaN,-0.094834,NaN,NaN,NaN,NaN,NaN,BMU,BM,NaN,High income
9012,Netherlands,1986,0.083348,0.211700,NaN,NaN,NaN,NaN,NaN,NLD,NL,NaN,High income
7024,Latvia,1974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LVA,LV,NaN,High income
10743,Sao Tome and Principe,2001,9.222037,NaN,NaN,15.000000,37.000000,15.73,15.73,STP,ST,Sub-Saharan Africa (excluding high income),Lower middle income
11223,Slovenia,2013,1.769201,1.600356,NaN,NaN,NaN,10.10,10.10,SVN,SI,NaN,High income
10981,Sierra Leone,1979,NaN,11.535712,-0.480306,7.250000,11.000000,NaN,NaN,SLE,SL,Sub-Saharan Africa (excluding high income),Low income
6415,Kenya,1989,13.789317,9.769009,6.815212,12.000000,17.250000,NaN,NaN,KEN,KE,Sub-Saharan Africa (excluding high income),Lower middle income
7708,Madagascar,1982,31.789705,31.906615,NaN,NaN,NaN,NaN,NaN,MDG,MG,Sub-Saharan Africa (excluding high income),Low income
9400,North Macedonia,2010,1.509975,2.041283,8.481425,7.466874,10.695838,32.02,32.02,MKD,MK,Europe & Central Asia (excluding high income),Upper middle income


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13832 entries, 0 to 13831
Data columns (total 13 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   country                                                              13832 non-null  object 
 1   year                                                                 13832 non-null  int64  
 2   Inflation, consumer prices (annual %)                                9582 non-null   float64
 3   Inflation, GDP deflator (annual %)                                   10981 non-null  float64
 4   Real interest rate (%)                                               4171 non-null   float64
 5   Deposit interest rate (%)                                            4587 non-null   float64
 6   Lending interest rate (%)                                            4413 non-null   float64
 7   Unem

## Actually creating the processing job now

In [34]:
# Create a SKLearn Processor
sklearn_processor = SKLearnProcessor(
    framework_version = "0.23-1",
    role=ROLE,
    instance_type="ml.t3.medium",
    instance_count=1
)

In [ ]:


sklearn_processor.run(
    code="preprocessing.py",
    inputs=[ProcessingInput(
        source="s3://"+S3_BUCKET+"/"+FILENAME,
        destination="/opt/ml/processing/input"
    )],
    outputs=[
        ProcessingOutput(
            source="/opt/ml/processing/train",
            output_name="train_data"
        ),
        ProcessingOutput(
            source="/opt/ml/processing/test",
            output_name="test_data"
        )
    ]
)


Job Name:  sagemaker-scikit-learn-2022-06-19-23-21-30-424
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://inflation-interest-unemployment/inflation interest unemployment.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-2-601163517885/sagemaker-scikit-learn-2022-06-19-23-21-30-424/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train_data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-eu-west-2-601163517885/sagemaker-scikit-learn-2022-06-19-23-21-30-424/output/train_data', 'LocalPath': '/opt/ml/processing/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputNam